In [3]:
!pip install -e model-tools/.

Obtaining file:///gpfs/data/tserre/irodri15/Brainscore/serre_repos/bs_hackathon/model-tools
  Preparing metadata (setup.py) ... done
     |█████████████▏                  | 362.0 MB 107.9 MB/s eta 0:00:05

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 881.9 MB 47 kB/s               
     |████████████████████████████████| 23.3 MB 28.3 MB/s            
     |████████████████████████████████| 412.3 MB 174 kB/s             
     |████████████████████████████████| 377 kB 93.8 MB/s            
     |████████████████████████████████| 50 kB 14.2 MB/s            
     |████████████████████████████████| 503 kB 100.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 3.8 MB 98.7 MB/s            
     |████████████████████████████████| 881.9 MB 43 kB/s               
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=79ae8e967dd500483772263438631fc4e4e314a72c8b2c753222a295e83f7c5d
  Stored in directory: /gpfs/home/irodri15/.cache/pip/wheels/19/a7/b9/0740c7a3a7d1d348f04823339274b90de25fbcd217b2ee1fbe
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
   

In [1]:
import json

from brainscore import score_model
from model_tools.brain_transformation import LayerMappedModel, TemporalIgnore
from candidate_models.model_commitments import brain_translated_pool, base_model_pool


Loading lookup from /gpfs/data/tserre/irodri15/Brainscore/venv_bs/lib/python3.6/site-packages/brainio_collection-0.1.0-py3.6.egg/brainio_collection/lookup.csv


/gpfs/data/tserre/irodri15/Brainscore/original_repos/brain-score/brainscore/metrics/__init__.py:37: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


ModuleNotFoundError: No module named 'candidate_models.model_commitments'

# Hackathon 2021 - Examples

This notebook provides a few starting points for the 2021 Brainscore Hackathon.

One thing to be aware of, is that the scores on the website are based on a "private" set of images and monkey data <sub> (= high variation set, variation 6, wider uniform distribution over transformation paramters, 2560 images) </sub>. Those aren't normally available to the public. The idea is that people can play around with "public" set instead, subit their model to the brainscore website, and they well use their private set to do the final evaluation. 

Luckily, Martin has kindly shared the private set with us, so we can replicate the website. **BUT IT MEANS SERRELAB IS NOT ALLOWED TO COMPETE IN THEIR RANKING!**

## The Brainscore Way

The brainscore way relies heavily on identifier strings. They use a string to select the model, but also one single string to decide which dataset, which visual area, and which fitting procedure to use.

For example, the code below should replicate alexnet's score for IT on the MajajHong2015 data.

In [ ]:
identifier = 'alexnet'
model = brain_translated_pool[identifier]
score = score_model(model_identifier=identifier, model=model, benchmark_identifier='dicarlo.MajajHong2015.IT-pls')
print(score)

The code will have first gone over all alexnet layers using the public set to select best layer. Then it used that layer for the final test with the private set. 

## Lore's Way

You can imagine that as soon as you want to run a couple of variations of fitting procedures, input images, etc., the identifier strings become very messy very quickly. In attempt to make things more flexible, I made a different kind of benchmark_identifiers. I also changed the score_model function to accept a bunch of kwargs. This new kind needs to start with "tol_". For example, I made "tol_imagedir" to flexibly choose which imagedir you want to extract activations for. The idea here is that the image filenames will provide the mapping between the new images and the images the monkeys saw.

Some caveats:
- The layer selection procedure will still use the brainscore defaults. It is best to select the layer to evaluate manually instead.
- You may want to disable result caching (already off by default for the hackathon) for this way, because the kwargs make the filenames for result saving way too long

For example, the code below should give the same results as the brainscor way for alexnet.

In [3]:
identifier = 'alexnet'
model = base_model_pool[identifier]  # notice it's base_model_pool now, not brain_translated_pool
region = 'IT'

# load info about which layer to select
with open("modelmeta-2019.json", 'r') as f:
    model_meta = json.load(f)
layer = [i['fields']['value'] for i in model_meta if
         i['model'] == 'benchmarks.ModelMeta' and
         i['fields']['key'] == region+'_layer' and
         i['fields']['model'] == identifier]
assert (len(layer) == 1)
layer = layer[0]

# layer -> region
model = LayerMappedModel(identifier + "__" + layer, activations_model=model, visual_degrees=8)

model.commit(region, layer)

# ignore time_bins
model = TemporalIgnore(model)

score = score_model(model_identifier=identifier,
                    model=model,
                    benchmark_identifier='tol_imagedir',  # benchmark that allows to flexibly set the imagedir
                    assembly_name='dicarlo.MajajHong2015.private',  # which neural recordings are we using
                    image_dir='image_dicarlo_hvm-private',  # based on which images' activations
                    baseline=True,  # using random splits (the brainscore way, as apposed to custom splits)
                    train_size=0.9,  
                    test_size=0.1,
                    baseline_splits=10,
                    region=region)

alexnet is accessed again and reloaded


TypeError: score_model() got an unexpected keyword argument 'assembly_name'

The imagedir does need to be in the os.getenv('BRAINIO_HOME', ~/brainio) directory for this to work. For the hackathon, this is ""/media/data_cifs_lrs/projects/prj_brainscore/hackaton2021/.brainio".